In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

In [27]:
#from google.colab import drive
#drive.mount('/content/drive')
torch.cuda.is_available()

False

In [28]:
def get_chest_images():
    TRANSFORM_IMG = transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()])
    file_path = "/storage/pipemon/6.867-xray-project/data/"
    #define train data loader
    trainset = datasets.ImageFolder(root=file_path+"train/", transform=TRANSFORM_IMG)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle=True, num_workers=2)
    #define test data loader
    valset = datasets.ImageFolder(root=file_path+"test/", transform=TRANSFORM_IMG)
    val_loader = torch.utils.data.DataLoader(valset, batch_size= 64, shuffle=False, num_workers=2)
    return train_loader, val_loader
    

In [29]:
class Chest_Disease_Net(nn.Module):
    def __init__(self):
        super(Chest_Disease_Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=16, stride=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=7, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(12544, 1024),
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(512,2)
         
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return(out)

In [ ]:
def run():
    # Parameters
    num_epochs = 10

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Chest_Disease_Net()
    model = model.to(device)

    trainloader, valloader = get_chest_images()

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    trainloader, valloader = get_chest_images()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(trainloader), 1):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print("Loss: " + str(running_loss/len(trainloader)))
        # save after every epoch
        torch.save(model.state_dict(), "model.%d" % epoch)

        model.eval()

        train_correct = 0
        train_total = 0
        with torch.no_grad():
            for data in tqdm(trainloader):
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                train_total += labels.size(0)

                train_correct += (predicted == labels).sum().item()
        print('Top One Error of the network on train images: %d %%' % (
                100 * (1 - train_correct / train_total)))


        correct = 0
        val_total = 0
        with torch.no_grad():
            for data in tqdm(valloader):
                images, labels = data

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                val_total += labels.size(0)

                correct += (predicted == labels).sum().item()

        print('Top One Error of the network on validation images: %d %%' % (
                100 * (1 - correct / val_total)))

        gc.collect()
run()


 22%|██▏       | 303/1352 [06:17<21:34,  1.23s/it]


 45%|████▍     | 605/1352 [12:30<15:01,  1.21s/it]


 67%|██████▋   | 907/1352 [18:43<09:14,  1.25s/it]


 89%|████████▉ | 1205/1352 [24:53<03:00,  1.23s/it]


100%|██████████| 1352/1352 [27:53<00:00,  1.24s/it]

  0%|          | 0/1352 [00:00<?, ?it/s]

...

 67%|██████▋   | 908/1352 [12:23<05:35,  1.32it/s]


 89%|████████▉ | 1206/1352 [16:28<01:48,  1.34it/s]


100%|██████████| 1352/1352 [18:28<00:00,  1.22it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

Top One Error of the network on train images: 36 %



 32%|███▏      | 128/400 [01:45<03:53,  1.17it/s]